In [10]:
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from common import ModelManager


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten: nn.Flatten = nn.Flatten()
        self.linear_relu_stack: nn.Sequential = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logistics = self.linear_relu_stack(x)
        return logistics


In [17]:
def run_training_model(train_data_loader: DataLoader, test_data_loader: DataLoader, model_params_path: str):
    model: NeuralNetwork = NeuralNetwork()
    model_manager: ModelManager = ModelManager(
        model=model,
        loss_fn=nn.CrossEntropyLoss(),
        optimizer=optim.SGD(model.parameters(), lr=1e-3),
    )
    epochs = 5
    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        model_manager.train(data_loader=train_data_loader)
        model_manager.test(data_loader=test_data_loader)
    model_manager.save_parameters(model_params_path)


def run_loading_model(test_data_loader: DataLoader, model_params_path: str):
    model: NeuralNetwork = NeuralNetwork()
    model_manager: ModelManager = ModelManager(
        model=model,
        loss_fn=nn.CrossEntropyLoss(),
        optimizer=optim.SGD(model.parameters(), lr=1e-3),
    )
    model_manager.load_parameters(model_params_path)
    model_manager.test(data_loader=test_data_loader)

In [18]:
if __name__ == '__main__':
    data_path: str = os.path.join("resources", "datasets", "fashion-mnist")
    model_params_path: str = os.path.join("resources", "models", "nn-fashion-mnist.pth")
    os.makedirs(data_path, exist_ok=True)
    os.makedirs(os.path.dirname(model_params_path), exist_ok=True)

    training_data: datasets.FashionMNIST = datasets.FashionMNIST(
        root=data_path,
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data: datasets.FashionMNIST = datasets.FashionMNIST(
        root=data_path,
        train=False,
        download=True,
        transform=ToTensor(),
    )

    train_data_loader = DataLoader(training_data, batch_size=64)
    test_data_loader = DataLoader(test_data, batch_size=64)

    for X, y in test_data_loader:
        print(f"Shape of X [N, C, H, W] {X.shape}")
        print(f"Shape of y {y.shape}, {y.dtype}")
        break

    run_training_model(train_data_loader, test_data_loader, model_params_path)
    run_loading_model(test_data_loader, model_params_path)

Shape of X [N, C, H, W] torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64]), torch.int64
Epoch 1
-------------------------------
loss: 2.305367 [    0/60000]
loss: 2.294987 [ 6400/60000]
loss: 2.278085 [12800/60000]
loss: 2.278510 [19200/60000]
loss: 2.255067 [25600/60000]
loss: 2.231116 [32000/60000]
loss: 2.241919 [38400/60000]
loss: 2.204893 [44800/60000]
loss: 2.196012 [51200/60000]
loss: 2.173568 [57600/60000]
Test Error: 
 Accuracy: 40.7%, Avg loss: 2.168902 

Epoch 2
-------------------------------
loss: 2.170758 [    0/60000]
loss: 2.169139 [ 6400/60000]
loss: 2.115000 [12800/60000]
loss: 2.132523 [19200/60000]
loss: 2.080977 [25600/60000]
loss: 2.024759 [32000/60000]
loss: 2.051031 [38400/60000]
loss: 1.974192 [44800/60000]
loss: 1.965565 [51200/60000]
loss: 1.906020 [57600/60000]
Test Error: 
 Accuracy: 56.9%, Avg loss: 1.907792 

Epoch 3
-------------------------------
loss: 1.927572 [    0/60000]
loss: 1.912810 [ 6400/60000]
loss: 1.800316 [12800/60000]
loss: 1.835579 [